In [17]:
import os
import cv2
import numpy as np

In [18]:
def measure_brightness(frame):
    """
    Measures the brightness of an image by calculating the average pixel intensity.

    Args:
        frame (np.array): Input image in BGR format.

    Returns:
        float: Average brightness of the image.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    brightness = np.mean(gray)  # Compute the mean intensity
    return brightness

In [19]:
def crop_image(frame, x, y, w, h):
    """
    Crops a region of interest (ROI) from an image.

    Args:
        frame (np.array): Input image.
        x (int): X-coordinate of the top-left corner of the ROI.
        y (int): Y-coordinate of the top-left corner of the ROI.
        w (int): Width of the ROI.
        h (int): Height of the ROI.

    Returns:
        np.array: Cropped image.
    """
    return frame[y:y+h, x:x+w]

In [20]:
def process_brightness_batch(input_path, output_path):
    """
    Processes a batch of images, measures brightness, and saves results.

    Args:
        input_path (str): Path to the folder containing input images.
        output_path (str): Path to save results.
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)  # Create the output directory if it doesn't exist

    best_frame = None
    best_frame_name = None
    highest_brightness = -1  # Initialize with a very low value

    brightness_scores = []

    for filename in os.listdir(input_path):
        input_file = os.path.join(input_path, filename)

        # Ensure the file is an image
        if not filename.lower().endswith(('.jpg')):
            continue

        # Read the image
        frame = cv2.imread(input_file)
        if frame is None:
            print(f"Failed to read {input_file}. Skipping...")
            continue
        
        cropped = crop_image(frame, x, y, w, h)
        cropped_frame = os.path.join(cropped_frame_path, f"{filename}")
        cv2.imwrite(cropped_frame, cropped)
        
        # Measure brightness
        brightness = measure_brightness(cropped)
        brightness_scores.append((filename, brightness))

        print(f"Processed {filename}: Brightness = {brightness:.2f}")

        # Save brightness scores to a text file
        scores_file = os.path.join(output_path, "brightness_scores.txt")
        with open(scores_file, "a") as f:
            f.write(f"Name {filename} : {brightness:.2f}\n")

        # Update the best frame if this one is better
        if brightness > highest_brightness:
            highest_brightness = brightness
            best_frame = frame  # Save the cropped frame as the best frame
            best_frame_name = filename
    
    # Save the best frame separately
    if best_frame is not None:
        best_frame_file = os.path.join(best_frame_path, f"best_frame_{best_frame_name}")
        cv2.imwrite(best_frame_file, best_frame)
        print(f"Best frame saved as: {best_frame_file} with Brightness = {highest_brightness}")


# Example usage
input_path = "Samples/0"   # Replace with your input images path
output_path = "Results/Bright/0"  # Replace with your results path
best_frame_path = "Results/Bright/0/Best"    # Replace with the path to save the best frame
cropped_frame_path = "Results/Bright/0/Crop" 
x, y, w, h = 120, 0, 400, 200  # Example crop rectangle (top-left at (100, 50), 200x200 size)
process_brightness_batch(input_path, output_path)

Processed frame_0000.jpg: Brightness = 130.57
Best frame saved as: Results/Bright/0/Best\best_frame_frame_0000.jpg with Brightness = 130.5666625
